In [1]:
import subprocess
import json
import pandas as pd

def describe_instances_to_dataframe():
    # Run AWS CLI command to describe instances
    command = "aws ec2 describe-instances"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print("Error:", result.stderr)
        return None

    # Parse JSON output
    try:
        instances_data = json.loads(result.stdout)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None

    # Extract relevant information for DataFrame
    instances_list = []
    for reservation in instances_data.get('Reservations', []):
        for instance in reservation.get('Instances', []):
            instance_info = {
                'InstanceId': instance.get('InstanceId', ''),
                'InstanceType': instance.get('InstanceType', ''),
                'State': instance.get('State', {}).get('Name', ''),
                'PublicIpAddress': instance.get('PublicIpAddress', ''),
                'PrivateIpAddress': instance.get('PrivateIpAddress', ''),
                # Add more fields as needed
            }
            instances_list.append(instance_info)

    # Create Pandas DataFrame
    df = pd.DataFrame(instances_list)
    return df

# Run the function
df = describe_instances_to_dataframe()

# Display the DataFrame
if df is not None:
    print(df)


            InstanceId InstanceType    State PublicIpAddress PrivateIpAddress
0  i-05eca5a0ba83f964b  m6a.2xlarge  running                     10.12.71.245
1  i-0e5cf592271fce67c     t3.large  running                     10.12.70.113
2  i-0ddcf0aef9b33b1d4  t3a.2xlarge  stopped                      10.12.70.26
3  i-024b7053f2d5e3876     t3.micro  stopped                       10.12.70.5
4  i-0a0f49cda5dae74ae  t3a.2xlarge  running                     10.12.71.205
5  i-0450ecc8cebe07fb8   m5d.xlarge  running                     10.12.65.136
6  i-0b4ec416daa9c1f44     t3.large  running                    172.16.19.223
7  i-0020d0abdde7f2293     t3.large  running                      172.16.6.67


In [8]:
import boto3

def get_instance_name(instance_id):
    # Specify the region
    region = 'us-west-2'  # Replace with your AWS region, e.g., 'us-east-1'

    # Create EC2 client
    ec2 = boto3.client('ec2', region_name=region)

    try:
        # Describe the instance to get its tags
        response = ec2.describe_instances(InstanceIds=[instance_id])

        # Extract the "Name" tag value
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                for tag in instance.get('Tags', []):
                    if tag['Key'] == 'Name':
                        return tag['Value']

        # If the instance has no "Name" tag
        return f"No 'Name' tag found for instance {instance_id}"

    except Exception as e:
        return f"Error: {str(e)}"

# Replace 'your-instance-id' with the actual instance ID
instance_id = 'i-05eca5a0ba83f964b'
instance_name = get_instance_name(instance_id)

print(f"The 'Name' tag for instance {instance_id} is: {instance_name}")


The 'Name' tag for instance i-05eca5a0ba83f964b is: nlcd-tbutzer-system-dev


In [10]:
for i,r in df.iterrows():
    print(r['InstanceId'])
    Name = get_instance_name(r['InstanceId'])
    print(Name)

i-05eca5a0ba83f964b
nlcd-tbutzer-system-dev
i-0e5cf592271fce67c
nlcd-pcluster-access-host
i-0ddcf0aef9b33b1d4
nlcd-kpostma-system-dev
i-024b7053f2d5e3876
nlcd-bschulz-system-dev
i-0a0f49cda5dae74ae
INL-pangeo
i-0450ecc8cebe07fb8
dev-nlcd-eks-ec2-cluster-lsis-system-nodes-persistent-a--eks_asg
i-0b4ec416daa9c1f44
HeadNode
i-0020d0abdde7f2293
HeadNode


In [11]:
df['InstanceName'] = df['InstanceId'].apply(get_instance_name)

In [12]:
df

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress,InstanceName
0,i-05eca5a0ba83f964b,m6a.2xlarge,running,,10.12.71.245,nlcd-tbutzer-system-dev
1,i-0e5cf592271fce67c,t3.large,running,,10.12.70.113,nlcd-pcluster-access-host
2,i-0ddcf0aef9b33b1d4,t3a.2xlarge,stopped,,10.12.70.26,nlcd-kpostma-system-dev
3,i-024b7053f2d5e3876,t3.micro,stopped,,10.12.70.5,nlcd-bschulz-system-dev
4,i-0a0f49cda5dae74ae,t3a.2xlarge,running,,10.12.71.205,INL-pangeo
5,i-0450ecc8cebe07fb8,m5d.xlarge,running,,10.12.65.136,dev-nlcd-eks-ec2-cluster-lsis-system-nodes-per...
6,i-0b4ec416daa9c1f44,t3.large,running,,172.16.19.223,HeadNode
7,i-0020d0abdde7f2293,t3.large,running,,172.16.6.67,HeadNode


In [13]:
selected_rows = df[df['InstanceName'].isin(['HeadNode', 'Compute'])]

In [14]:
selected_rows

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress,InstanceName
6,i-0b4ec416daa9c1f44,t3.large,running,,172.16.19.223,HeadNode
7,i-0020d0abdde7f2293,t3.large,running,,172.16.6.67,HeadNode
